In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import os
import random
#from keras import backend as K
import timeit
import os
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.contrib.eager as tfe


Using TensorFlow backend.


In [2]:

tf.enable_eager_execution()

print("TensorFlow version: {}".format(tf.VERSION))
print("Eager execution: {}".format(tf.executing_eagerly()))

TensorFlow version: 1.8.0
Eager execution: True


In [3]:
class ColorModel(tf.keras.Model):
    def __init__(self):
        super(ColorModel, self).__init__()
        self.layer1 = tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same', strides=2)
        self.layer2 = tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding='same')
        self.layer3 = tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding='same', strides=2)
        self.layer4 = tf.keras.layers.Conv2D(256, (3,3), activation='relu', padding='same')
        self.layer5 = tf.keras.layers.Conv2D(256, (3,3), activation='relu', padding='same', strides=2)
        self.layer6 = tf.keras.layers.Conv2D(512, (3,3), activation='relu', padding='same')
        self.layer7 = tf.keras.layers.Conv2D(512, (3,3), activation='relu', padding='same')
        self.layer8 = tf.keras.layers.Conv2D(256, (3,3), activation='relu', padding='same')
        self.layer9 = tf.keras.layers.RepeatVector(32 * 32)
        self.layer10 = tf.keras.layers.Reshape(([32, 32, 1536]))
#        layer11 = concatenate([encoder_output, fusion_output], axis=3) 
        self.layer12 = tf.keras.layers.Conv2D(256, (1, 1), activation='relu', padding='same')

        #Decoder
        self.layer13 = tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding='same')
        self.layer14 = tf.keras.layers.UpSampling2D((2, 2))
        self.layer15 = tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same')
        self.layer16 = tf.keras.layers.UpSampling2D((2, 2))
        self.layer17 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same')
        self.layer18 = tf.keras.layers.Conv2D(16, (3,3), activation='relu', padding='same')
        self.layer19 = tf.keras.layers.Conv2D(2, (3, 3), activation='tanh', padding='same')
        self.layer20 = tf.keras.layers.UpSampling2D((2, 2))
        
    def call(self, input1,input2):
        """Run the model."""
        result = self.layer1(input1)
        result = self.layer2(result)
        result = self.layer3(result)
        result = self.layer4(result)
        result = self.layer5(result)
        result = self.layer6(result)
        result = self.layer7(result)
        result = self.layer8(result)
        result1=self.layer9(input2)
        result1=self.layer10(result1)
        result=tf.keras.layers.concatenate([result,result1])

        result = self.layer12(result)
        result = self.layer13(result)
        result = self.layer14(result)
        result = self.layer15(result)
        result = self.layer16(result)
        result = self.layer17(result)
        result = self.layer18(result)
        result = self.layer19(result)
        result = self.layer20(result)
        return result



In [4]:
model=ColorModel()

In [5]:
train_path="Train/"
folders=os.listdir(train_path)
files=[]
for f in folders:
    files.extend([train_path+f+"/"+each for each in os.listdir(train_path+f)])
random.shuffle(files)

val=files[-100:]
files=files[:-100]

In [6]:
def loss(model, x,x1, y):
    y_ = model(x,x1)
    return tf.losses.mean_squared_error(labels=y, predictions=y_)


def grad(model, inputs,embed, targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs,embed, targets)
    return tape.gradient(loss_value, model.variables)

In [21]:
optimizer = tf.train.AdagradOptimizer(learning_rate=0.01)

In [14]:
embeddings=np.load("test.npy")
names=np.load('test1.npy')

In [15]:
def image_gen():
    global files
    i=0
    while True:
        if(i>len(files) or i+1>len(files)):
            i=0
        x,x1,y=read_images(files[i:i+1])
        x=x[0]
        x1=x1[0]
        y=y[0]
        i+=1
        yield x,x1,y
        
def read_images(files):
    X = []
    embed = []
    for filename in files:
        X.append(img_to_array(load_img(filename)))
        embed.append(embeddings[np.argwhere(names == filename)[0][0]])

    embed = np.array(embed, dtype=float)
    X = np.array(X, dtype=float)/255
    grayscaled_rgb = gray2rgb(rgb2gray(X))
    lab_batch = rgb2lab(X)
    X_batch = lab_batch[:,:,:,0]
    X_batch = X_batch.reshape(X_batch.shape+(1,))
    Y_batch = lab_batch[:,:,:,1:] / 128
    return (X_batch,embed, Y_batch)

val_x,val_x1,val_y = read_images(val)


In [18]:
def test(model,i):
    color_me = []
    color_me.append(img_to_array(load_img(files[0])))
    color_me = np.array(color_me, dtype=float)
    gray_me = gray2rgb(rgb2gray(1.0/255*color_me))
    color_me_embed =embeddings[np.argwhere(names == files[0])[0][0]]
    color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
    color_me = color_me.reshape(color_me.shape+(1,))


    # Test model
    color_me_embed=np.expand_dims(color_me_embed,axis=0)
    output = model(tf.convert_to_tensor(color_me,dtype=tf.float32), tf.convert_to_tensor(color_me_embed,dtype=tf.float32))
    output = output * 128

    # Output colorizations:
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[0][:,:,0]
    cur[:,:,1:] = output[0]
    imsave("result/img_0"+str(i)+".png", lab2rgb(cur))

In [23]:
train_loss_results = []
train_accuracy_results = []
#gen=image_gen(20)
num_epochs = 100
ds = tf.data.Dataset.from_generator(
    image_gen, (tf.float32, tf.float32, tf.float32))
value = ds.batch(20)
checkpoint = tfe.Checkpoint(model=model)

checkpoint.restore(tf.train.latest_checkpoint("./model"))
for epoch in range(num_epochs):
    epoch_loss_avg = tfe.metrics.Mean()
    #epoch_accuracy = tfe.metrics.Accuracy()
    i=0
    
    start_time = timeit.default_timer()
# code you want to evaluate
    for x,x1,y in value:
    # Optimize the model
        if i==20:
            break
        grads = grad(model,x,x1, y)
        optimizer.apply_gradients(zip(grads, model.variables),
                              global_step=tf.train.get_or_create_global_step())
        i+=1
        epoch_loss_avg(loss(model, x,x1, y))  # add current batch loss
   
    elapsed = timeit.default_timer() - start_time
    print("epoch time ",elapsed)
    if epoch%10==0:
        start_time = timeit.default_timer()
        test(model,epoch)
        elapsed = timeit.default_timer() - start_time
        print("test time ",elapsed)
    if(epoch%10==0):
        checkpoint.save(os.path.join("./model", "ckpt"))
    train_loss_results.append(epoch_loss_avg.result())
    #train_accuracy_results.append(epoch_accuracy.result())
    print("Epoch {:03d}: Loss: {:.9f}".format(epoch,epoch_loss_avg.result()))

epoch time  28.80893547499727
test time  0.11766852899745572


/usr/local/lib/python3.5/dist-packages/skimage/util/dtype.py:130: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


Epoch 000: Loss: 0.002505680
epoch time  28.10058647299593
Epoch 001: Loss: 0.002500428
epoch time  28.034092877001967
Epoch 002: Loss: 0.002498059
epoch time  28.089152532993467
Epoch 003: Loss: 0.002496596
epoch time  28.433253272
Epoch 004: Loss: 0.002495537
epoch time  28.07846629100095
Epoch 005: Loss: 0.002494689
epoch time  28.11818461600342
Epoch 006: Loss: 0.002493967
epoch time  28.08062624100421
Epoch 007: Loss: 0.002493328
epoch time  28.136538174003363
Epoch 008: Loss: 0.002492739
epoch time  28.0965453809913
Epoch 009: Loss: 0.002492189
epoch time  28.14839293299883
test time  0.11805944399384316
Epoch 010: Loss: 0.002491673
epoch time  28.431979091998073
Epoch 011: Loss: 0.002491176
epoch time  28.138254727004096
Epoch 012: Loss: 0.002490698
epoch time  29.241184621991124
Epoch 013: Loss: 0.002490240
epoch time  28.601219421994756
Epoch 014: Loss: 0.002489791
epoch time  28.09032566600945
Epoch 015: Loss: 0.002489354
epoch time  28.116758763993857
Epoch 016: Loss: 0.0024